In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
from package import embeddings as ebd

In [6]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import ollama
import chromadb

In [7]:
embeddings = OllamaEmbeddings(model="mistral")

In [8]:
persist_directory = "./vectordb"

In [9]:
def get_docs_from_url(url):
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict()
    )
    text = loader.load()
    return text

def create_documents(text):    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.split_documents(text)
    return documents

def add_to_vectordb(documents, embeddings, persist_directory=None, collection_name=None):
    db = Chroma.from_documents(
        documents=documents, 
        embedding=embeddings, 
        persist_directory=persist_directory, 
        collection_name=collection_name
    )
    return db

def load_vectordb(embeddings, persist_directory=None, collection_name=None):
    vectordb = Chroma(
        embedding_function=embeddings,
        persist_directory=persist_directory, 
        collection_name=collection_name, 
    )
    return vectordb

def format_context(context):
    # text = "\n\n"
    text = "\n"
    context = text.join(cntx.page_content for cntx in context)
    return context

def create_prompt(query, vectordb):
    context = vectordb.as_retriever().invoke(query)
    context = format_context(context)
    # prompt = f"Question: {query}\n\nContext: {context}"
    prompt = f"Query: {query}\n\nContext: {context}"
    return prompt

def ask_llm(prompt):
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

In [13]:
ollama.chat(messages=[{'role': 'user', 'content': "Query: Hi"}])

RequestError: must provide a model

In [11]:
dir(ollama)

['AsyncClient',
 'ChatResponse',
 'Client',
 'GenerateResponse',
 'Message',
 'Options',
 'ProgressResponse',
 'RequestError',
 'ResponseError',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_client',
 '_types',
 'chat',
 'copy',
 'create',
 'delete',
 'embeddings',
 'generate',
 'list',
 'pull',
 'push',
 'show']

In [19]:
def add_website_data_to_vectordb(url, embeddings, persist_directory=None, collection_name=None):
    text = get_docs_from_url(url)
    documents = create_documents(text)
    add_to_vectordb(embeddings=embeddings, documents=documents, persist_directory=persist_directory, collection_name=collection_name)

In [7]:
text = get_docs_from_url("https://ccp.net.au/the-problem-with-pop-psychology/")
documents = create_documents(text)

In [8]:
add_to_vectordb(embeddings=embeddings, documents=documents, persist_directory=persist_directory, collection_name="pop_psychology")

In [9]:
vectordb = load_vectordb(embeddings=embeddings, persist_directory=persist_directory, collection_name="pop_psychology")

In [13]:
query = "is pop psychology a bad thing?"
prompt = create_prompt(query, vectordb=vectordb)
print(prompt)

Query: is pop psychology a bad thing?

Context: sexual assault, childhood sexual assault and domestic violence, those involved in motor vehicle accidents, soldiers, war veterans/peace keepers, police members, and emergency services workers. Jon also has extensive experience working with people experiencing depression, anxiety and adjustment disorders, and those suffering from substance use difficulties.
Jon Finch is a clinical psychologist who specialises in the treatment of Posttraumatic Stress Disorder (PTSD). He is a skilled psychologist with 20 years experience. Jon is a certified Cognitive Processing Therapy (CPT) for PTSD trainer. Jon is also a certified, individual Schema Therapist (standard level) with the International Society of Schema Therapy. Jon began his career working with soldiers, war veterans/peace keepers at the Veteran and Veterans Families Counselling Service, in the role of counsellor and Deputy Director. He then worked for Davidson Trahaire Corpsych as the Victor

In [14]:
result = ask_llm(prompt)
print(result)

 Based on the information provided, it appears that Jon Finch, a clinical psychologist with extensive experience in treating PTSD and other mental health conditions, expresses caution towards the potential negative effects of "pop psychology." Pop psychology can trivialize mental health conditions through the use of simplistic language and oversimplified explanations. This can make it difficult for individuals to understand the complexity of their mental health issues and may discourage them from seeking professional help.

To avoid the negative effects of pop psychology, Jon Finch advises considering the reliability of information sources. One should ensure that the information comes from reputable psychologists with references to supporting research. Additionally, critical thinking skills are essential in discerning accurate psychological information from misinformation. Lastly, professional help from a qualified mental health professional is crucial for those experiencing mental hea

In [15]:
query = "how many bad effects does pop psychology have on people?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=vectordb
        )
    )
)

 Pop psychology can have several negative effects on people, particularly those who have experienced trauma or mental health conditions. One of the main issues is that pop psychology terminology can trivialize mental health conditions and make it difficult for people to get the help they need. This is especially important for individuals who have experienced traumatic events such as sexual assault, childhood sexual assault, domestic violence, motor vehicle accidents, or war-related experiences.

It's essential to be cautious about where we get our information on psychology and mental health. We should ensure that the sources are reliable, written by qualified psychologists, and backed up by research. Additionally, we must consider the transparency and accountability of the organizations publishing the content and their investment in psychological research.

Critical thinking is also an essential tool to avoid the negative effects of pop psychology. If psychological advice seems too goo

In [17]:
query = "how many bad effects does pop psychology have on people?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=vectordb
        )
    )
)

 Pop psychology can have several negative effects on people, particularly those who have experienced traumatic events such as sexual assault, childhood sexual assault, domestic violence, motor vehicle accidents, and military service. One of the main negative effects is that pop psychology terminology can trivialize mental health conditions, making it difficult for people to get the help they need. This can be especially problematic for individuals with conditions like PTSD, which are characterized by debilitating distress.

To avoid some of these negative effects, it's important to consider where your information about psychology comes from. Reliable sources include books, blogs, articles, and social media content written by psychologists with references to research that supports their claims. Reputable organizations publishing this content should have transparent and accountable processes and be invested in psychological research.

Critical thinking is also a great way to avoid the ne

In [18]:
query = "list the bad effect of pop psychology along with examples"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=vectordb
        )
    )
)

 Pop psychology, which refers to psychological concepts and ideas that are widely disseminated through popular culture rather than through academic or scientific channels, can have several negative effects, particularly in the context of sensitive and complex issues such as sexual assault, childhood sexual assault, domestic violence, PTSD, and mental health in general. Some of these negative effects include:

1. Lack of precision and misunderstanding: Pop psychology terms are often used imprecisely or too broadly, leading to confusion and a lack of understanding about the true nature of mental health conditions or responses to traumatic experiences. For example, using the term PTSD (Post-Traumatic Stress Disorder) casually to describe any kind of stress response can dilute its significance and make it difficult for individuals to seek appropriate help.
2. Oversimplification: Pop psychology can oversimplify complex psychological processes, reducing them to easy-to-digest soundbites or c

In [23]:
query = "list the bad effect of pop psychology along with examples"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=load_vectordb(embeddings=embeddings, persist_directory=persist_directory, collection_name="pop_psychology")
        )
    )
)

 Pop psychology can have several negative effects, especially when it comes to understanding complex issues such as PTSD, trauma, and mental health in general. Here are some potential downsides:

1. Lack of Precision: Pop-psychology terms often lack precision, which can lead to misunderstandings and confusion. For instance, the term "PTSD" is used to describe a specific response to traumatic experiences. However, if this term is used too broadly, it may dilute its significance, leading to a lack of understanding about the actual behaviors involved.
2. Simplistic Solutions: Pop psychology often offers simplistic solutions to complex problems. For example, telling someone who has experienced trauma that they just need to "move on" or "get over it" can be harmful and dismissive. These statements do not take into account the complexity of emotional experiences and the healing process.
3. Misinformation: Pop psychology can spread misinformation about mental health issues, which can lead to 

In [21]:
add_website_data_to_vectordb(
    url="https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html",
    embeddings=embeddings,
    persist_directory=persist_directory,
    collection_name="rag_llm",
)

In [22]:
add_website_data_to_vectordb(
    url="https://aws.amazon.com/bedrock/?refid=1325fdf1-f38b-4c89-8bf9-dc39eeb87ff8",
    embeddings=embeddings,
    persist_directory=persist_directory,
    collection_name="rag_llm",
)

In [24]:
add_website_data_to_vectordb(
    url="https://www.promptingguide.ai/research/rag",
    embeddings=embeddings,
    persist_directory=persist_directory,
    collection_name="rag_llm",
)

In [25]:
query = "summarize the benefits of retrival augmented generation along with its examples"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=load_vectordb(embeddings=embeddings, persist_directory=persist_directory, collection_name="rag_llm")
        )
    )
)

 Retrieval-Augmented Generation (RAG) and fine-tuning are two distinct approaches in the field of Large Language Models (LLMs), each with its unique benefits. Let's summarize the advantages of RAG along with some examples:

Benefits of Retrieval-Augmented Generation:

1. New Knowledge Integration: RAG systems can integrate new knowledge by retrieving information from external data sources and incorporating it into the model's responses, making them suitable for applications requiring up-to-date or specialized information.
2. Customized Responses: By retrieving relevant information, RAG models can generate customized responses that cater to specific user queries or contexts, enhancing the overall user experience.
3. Scalability and Adaptability: RAG systems can be more scalable and adaptable than fine-tuned models as they don't require large amounts of labeled data for adapting to new domains or tasks, making them suitable for various applications without extensive retraining.
4. Flexib

In [26]:
query = "what is the worst effect of pop psychology?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=load_vectordb(embeddings=embeddings, persist_directory=persist_directory, collection_name="rag_llm")
        )
    )
)

 I'm assuming you are asking about the potential negative effects of pop psychology, not about pop psychology in relation to RAG systems. Pop psychology refers to psychological theories and applications that are widely popularized but may lack scientific rigor or be oversimplified. Here are some possible negative effects of pop psychology:

1. Misinformation and misunderstanding: Pop psychology can spread misinformation and misunderstanding about psychological concepts, which can lead to confusion, stigma, or harmful beliefs. For example, the belief that people have a "left brain" for analytical thinking and a "right brain" for creativity is a common pop psychology myth with no scientific basis.
2. Oversimplification: Pop psychology can oversimplify complex psychological phenomena, reducing them to simplistic models or catchphrases that do not capture the nuances and complexity of human behavior. For example, labeling someone as an "introvert" or an "extrovert" based on their social be

In [27]:
query = "Do you know who is uncel chub?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=load_vectordb(embeddings=embeddings, persist_directory=persist_directory, collection_name="rag_llm")
        )
    )
)

 I apologize for the confusion, but "Uncel Chub" does not appear to be mentioned or related to the context provided in your query, which is about Retrieval-and-Generation (RAG) systems. If you meant to ask about a specific person or term in a different context, please provide more details and I'll do my best to help answer any question you might have.


In [28]:
query = "summarize the benefits of retrival augmented generation along with its examples"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=load_vectordb(embeddings=embeddings, persist_directory=persist_directory, collection_name="pop_psychology")
        )
    )
)

 Regarding your specific query about Retrieval Augmented Generation (RAG) and its benefits with examples, I'd be happy to help you with that. However, since there is no mention of RAG in the context provided in your text, I will provide a general explanation of RAG and its benefits based on psychological research.

Retrieval Augmented Generation (RAG) is a subfield of artificial intelligence that combines retrieval models with generative models to produce more accurate and contextually relevant responses. In the context of mental health and psychology, RAG can be beneficial in several ways:

1. Improved Accuracy and Precision: RAG systems can help provide more precise and accurate information about specific psychological concepts, such as PTSD or depression, by using retrieval models to access up-to-date research and evidence-based interventions. This can be especially important in complex areas like mental health, where nuance and precision are essential.
2. Personalized Recommendatio

In [31]:
query = "summarize the benefits of retrival augmented generation along with its examples"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='langchain',
            )
        )
    )
)

 Retrieval-Augmented Generation (RAG) is a model architecture that combines text retrieval and generative models to improve the performance and flexibility of language models. RAG allows models to access external knowledge from large datasets or the web, enabling them to generate more accurate, contextually relevant, and diverse responses.

Benefits of Retrieval-Augmented Generation:

1. Improved accuracy: By retrieving relevant information from external data sources, RAG can help language models provide more accurate and detailed responses, especially for complex queries or questions requiring specific knowledge.
2. Contextual understanding: RAG enables models to access contextually relevant data, making them better at understanding the nuances of various situations and generating appropriate responses.
3. Diversity and creativity: By providing a wider range of input examples from external data sources, RAG can help language models generate more diverse and creative responses, leading

In [32]:
query = "what is the worst effect of pop psychology?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='langchain',
            )
        )
    )
)

 Pop psychology, also known as popular psychology or self-help psychology, refers to psychological concepts and ideas that have become widely known and disseminated through various media channels such as books, magazines, television shows, and social media. While pop psychology can provide valuable insights and practical tools for improving mental health and well-being, it can also have negative effects. Here are some of the worst effects of pop psychology:

1. oversimplification of complex psychological issues: Pop psychology often presents simplistic solutions to complex psychological problems, which can be misleading and ineffective. This oversimplification can lead people to believe that they can easily solve their mental health issues by following a few simple steps or techniques without the need for professional help.
2. stigmatization of mental health conditions: Pop psychology can sometimes perpetuate negative stereotypes about mental health conditions, making it more difficult

In [33]:
query = "what is bedrock?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='langchain',
            )
        )
    )
)

 Bedrock refers to the solid, compacted layer of rock that forms the base of the earth's surface. It provides a solid foundation for the soil above and supports the structures built on it, such as buildings and roads. Bedrocks can be made up of various types of rocks, including igneous, sedimentary, or metamorphic, and may contain fossils or other natural features. In geology, studying bedrock is an important aspect of understanding the underlying structure and history of an area.


In [34]:
query = "what is bedrock?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='rag_llm',
            )
        )
    )
)

 Bedrock refers to the foundational data or knowledge that a language model (LLM) like a Retrieval-Augmented Generation (RAG) system is built upon. It's essentially the collection of facts, concepts, and information that the LLM uses as a base for generating responses or understanding user queries. In the context provided in your text, it seems that RAG models use pretrained language models like sentence transformers or text-embedding-ada-002 as their bedrock knowledge and then retrieve additional relevant information to generate accurate and customized responses. Fine-tuning, on the other hand, is a different approach where you train a model on a specific dataset to improve its performance on that particular task. Both methods can be used in an iterative process to enhance the capabilities of LLMs for complex applications.


In [36]:
type(documents[0])

langchain_core.documents.base.Document

In [37]:
from langchain_core.documents.base import Document

In [39]:
documents[0].dict()

{'page_content': 'The Problem with Pop Psychology\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content \n\n\nMenu \n(03) 9077 0122\nENTER TELEHEALTH APPOINTMENT\n \n\n\n\n\n\n\n\n \n\nMenu \n\n\n\n\nMenu \nHOME\nABOUT\n\nOur Team\nFAQ\nWork With Us\n\n\nSERVICES\n\nTrauma\nPregnancy & Birth\nNew Parents\nAnxiety\nDepression\nPTSD\nADHD Assessment for Adults\nAbout Cognitive Processing Therapy for PTSD\nTelehealth\n\n\nFEES\nHOW TO REFER\nCLINICIAN TRAINING\nBLOG\nCONTACT\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe Problem with Pop Psychology \n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n \n\n\n \n\n\t\t\t\t\t\t\t\t\t\tJon Finch\t\t\t\t\t\n\n\n\n \n\nNovember 14, 2023',
 'metadata': {'source': 'https://ccp.net.au/the-problem-with-pop-psychology/',
  'title': 'The Problem with Pop Psychology',
  'description': "In today's fast-paced world, the language we use often reflects the latest trends and ideas. One area whe

In [40]:
uncle_chub = Document(
    page_content="Uncle chub is a former guild leader of JointBoi from the game called ROO",
    metadata={
        "source": 'https://unclechub.com/jointboi',
        "title": 'JointBoi ftw',
        "description": "",
        "language": 'en-AU'
    }
)

roo = Document(
    page_content="A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.",
    metadata={
        "source": 'https://roo.com/about',
        "title": 'Ragnarok Online Origin',
        "description": "",
        "language": 'en-AU'
    }
)

milkyway = Document(
    page_content="MilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.",
    metadata={
        "source": 'https://milkyway/alliance',
        "title": 'We are MilkyWay!!!',
        "description": "",
        "language": 'en-AU'
    }
)

add_to_vectordb(embeddings=embeddings, documents=[uncle_chub, roo, milkyway], persist_directory=persist_directory, collection_name="roo")

In [41]:
query = "Have you heard Uncle Chub before?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='roo',
            )
        )
    )
)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


 Yes, I have heard of UncleChub as he was formerly the guild leader of MilkyWay (previously known as JointBoi) in Ragnarok Online Origin. He has been replaced by Wennedy as the current guild leader.


In [42]:
query = "What does the ROO look like?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='roo',
            )
        )
    )
)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


 I'm sorry for any confusion, but it seems there are multiple acronyms being used in your query that could potentially stand for different things: "ROO," "Ragnarok Online Origin," and "ROO" being mentioned as a game where Uncle Chub was a former guild leader.

Without more context or information, it is impossible to answer what "ROO" specifically looks like as it could refer to a game, a guild name, or something else entirely. If you meant to ask about the visual appearance of Ragnarok Online Origin or any of its related in-game elements, I would recommend checking out the official website, trailers, or screenshots for more information.

As for the term "ROO," it is often used in gaming communities as an acronym for "Realm of Obtuse" or "Ruled by Obsessives," referring to a game's community that can be particularly intense or dedicated. Again, without more context, it is unclear if this is what you meant by your query.

If you meant to ask about the appearance of MilkyWay guild in Ragn

In [44]:
ragnarok_online_origin = Document(
    page_content="Ragnarok Online Origin or so called ROO is still open for the global service.",
    metadata={
        "source": 'https://roo.com/main',
        "title": 'Ragnarok Online Origin',
        "description": "",
        "language": 'en-AU'
    }
)

add_to_vectordb(embeddings=embeddings, documents=[ragnarok_online_origin], persist_directory=persist_directory, collection_name="roo")

In [45]:
query = "What does the ROO look like?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='roo',
            )
        )
    )
)

 I'm an AI language model and I don't have the ability to see or describe physical objects like the ROO (Ragnarok Online Origin) in real life. The term "ROO" refers to a specific version or iteration of the massively multiplayer online role-playing game (MMORPG) Ragnarok Online. As for its appearance, it's important to note that MMORPGs are virtual games and their graphics, interface, and overall design can vary greatly depending on the developer's vision and updates over time. There isn't any specific information provided in your context about what the ROO looks like. If you have more details or specifications about the game's graphics or user interface, I would be happy to help answer that question!

As for MilkyWay being a guild from ROO and UncleChub being a former guild leader of another ROO guild (JointBoi), those are just pieces of information about the context of the game and its community.


In [46]:
query = "Poor people should play ROO or not?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='roo',
            )
        )
    )
)

 It ultimately depends on your personal circumstances and budget for gaming. Ragnarok Online Origin (ROO) being a classic MMORPG with a top-up system may require regular financial investment to fully enjoy the game. If you are currently experiencing financial hardship, it might be best to consider other free or low-cost gaming options. However, if you have a stable budget for gaming and can afford the costs associated with ROO, then it could still be an enjoyable experience.

Joining a guild like MilkyWay, which has an experienced leader in Wennedy, could potentially help you get acclimated to the game and make your experience more enjoyable. But ultimately, only you can decide if the investment of time and money is worth it for you.


In [47]:
query = "Do you have any information about ROO?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='roo',
            )
        )
    )
)

 I. Introduction
Ragnarok Online Origin, often referred to as ROO, is a Massively Multiplayer Online Role-Playing Game (MMORPG) that has been in operation for the global audience. The game is known for its top-up systems which can potentially drain players' financial resources if not managed carefully. MilkyWay is one of the guilds within this game, and they have experienced a leadership change from UncleChub to Wennedy. UncleChub was previously the guild leader of another ROO guild named JointBoi.

II. Ragnarok Online Origin (ROO)
1. Overview: Ragnarok Online Origin, often referred to as just ROO, is a classic MMORPG that continues to operate globally. The game has gained notoriety for its top-up systems which can be quite costly for players.
2. Gameplay: Ragnarok Online Origin offers an immersive gaming experience with intricate character development and strategic teamwork in various PvE and PvP content. Players explore a vast world filled with monsters, quests, and interactive commu

In [43]:
query = "Who is MilkyWay's guild leader now?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='roo',
            )
        )
    )
)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


 The current guild leader of MilkyWay in Ragnarok Online Origin is Wennedy. UncleChub was previously the guild leader but has since stepped down from that role.


In [48]:
query = "WHO IS RATTIKARN LAPSONGPHON:"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='roo',
            )
        )
    )
)

 Rattikarn Lapsongphon is not mentioned in the provided context regarding MilkyWay guild in Ragnarok Online Origin and UncleChub's previous guild, JointBoi.


In [50]:
query = "do you know anotonio petrucci?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='langchain',
            )
        )
    )
)

 Antonio Petrucci is a well-known figure in the world of classical guitar. He was born on December 12, 1961, in Italy and began his musical studies at a young age. Petrucci is renowned for his exceptional technique, expressive playing, and comprehensive knowledge of classical guitar repertoire.

Throughout his career, he has been an active performer, competition juror, teacher, and composer. Some of his notable accomplishments include winning first prize in the 1985 International Guitar Competition in Israel, releasing numerous recordings, and publishing various educational materials for classical guitar.

Petrucci's teaching career includes positions at prestigious institutions such as the Royal Conservatory of Brussels and the Gnessin State Special Music School in Moscow. He is also known for his masterclasses and private lessons given around the world.

Additionally, Antonio Petrucci has contributed significantly to the classical guitar community through his compositions and arrange

In [51]:
query = "คนไทยเป็นคนยังไง?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='langchain',
            )
        )
    )
)

 Thai people are friendly, hospitable, and respectful. They place great importance on family and community, and their culture is rich in history, traditions, and beautiful art. Thai cuisine is world-renowned for its unique flavors and ingredients. Overall, Thai people are known for their warmth, kindness, and welcoming nature to visitors from all over the world.


In [52]:
query = "Who is pathompol nilchaikovit?"

print(
    ask_llm(
        create_prompt(
            query=query, 
            vectordb=Chroma(
                embedding_function=embeddings,
                persist_directory=persist_directory, 
                collection_name='langchain',
            )
        )
    )
)

 Pathompol Nilchaikovit is a Thai professional footballer who plays as a midfielder. He was born on December 2, 1986, in Thailand. Nilchaikovit started his professional football career with Muangthong United in the Thai Premier League. He has also played for other Thai clubs, including Buriram United and Chiangrai United.

In addition to his domestic successes, Nilchaikovit has represented Thailand at the international level, making his debut for the national team in 2010. He was a member of the squad that reached the quarter-finals of the 2014 AFC Asian Cup.

Nilchaikovit is known for his impressive passing abilities and work rate in the midfield. He has won numerous awards throughout his career, including the Thai Premier League Player of the Year award in 2017 and the AFC Champions League top scorer award in 2013.


In [15]:
1+1

2

In [4]:
from package.crud.read_vectordb import load_vectordb
from package.utils import get_context, format_context, generate_prompt
from package.generate_response import ask_llm

In [5]:
vectordb = load_vectordb()

C:\Users\doublebank\.conda\envs\local-rag-llm\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [6]:
query = "Uncle Chub"

In [7]:
%%time
context = get_context(query, vectordb)

CPU times: total: 15.6 ms
Wall time: 4.92 s


In [9]:
context = format_context(context)

In [10]:
prompt = generate_prompt(query, context)

In [13]:
print(prompt)

Query: Uncle Chub

Context: MilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.
A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.
Uncle chub is a former guild leader of JointBoi from the game called ROO
Ragnarok Online Origin or so called ROO is still open for the global service.


In [15]:
ask_llm(prompt)

' UncleChub was formerly the guild leader of "JointBoi" in Ragnarok Online Origin (ROO), but he has since been replaced by Wennedy as the current guild leader of "MilkyWay". If you\'re playing ROO and looking for a new guild, keep in mind that MilkyWay is currently being led by Wennedy. Additionally, be aware that Ragnarok Online Origin, like many other classic MMORPGs with new top-up systems, can be financially draining.'

In [17]:
import ollama

In [19]:
dir(ollama)

['AsyncClient',
 'ChatResponse',
 'Client',
 'GenerateResponse',
 'Message',
 'Options',
 'ProgressResponse',
 'RequestError',
 'ResponseError',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_client',
 '_types',
 'chat',
 'copy',
 'create',
 'delete',
 'embeddings',
 'generate',
 'list',
 'pull',
 'push',
 'show']

In [21]:
from langchain_community.llms import Ollama

llm = Ollama(base_url = 'http://localhost:11434', model="mistral")

In [ ]:
llm.invoke()

In [24]:
dir(llm)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [23]:
llm.invoke(prompt)

' It seems that "Uncle Chub" was formerly the guild leader of "JointBoi" in the game Ragnarok Online Origin (ROO). Now, the guild leader position has been taken over by someone named Wennedy, and MilkyWay is a different guild.\n\nThe context does not provide any information on why Uncle Chub was no longer the guild leader or what happened to JointBoi. It also does not mention anything about the game draining money or being a classic MMORPG with top-up systems. These details appear unrelated to Uncle Chub and his former guild.'

In [ ]:
llm

In [ ]:
ollama.chat()

In [25]:
from langchain_community import vectorstores

In [28]:
dir(vectorstores.chroma.VectorStore)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_asimilarity_search_with_relevance_scores',
 '_cosine_relevance_score_fn',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_texts',
 'adelete',
 'afrom_documents',
 'afrom_texts',
 'amax_marginal_relevance_search',
 'amax_marginal_relevance_search_by_vector',
 'as_retriever',
 'asearch',
 'asimilarity_search',
 'asimilarity_search_by_vector',
 'asimila

In [78]:
import chromadb
client = chromadb.PersistentClient(path="./vectordb")

In [30]:
dir(client)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add',
 '_admin_client',
 '_count',
 '_create_system_if_not_exists',
 '_delete',
 '_get',
 '_get_identifier_from_settings',
 '_identifer_to_system',
 '_identifier',
 '_modify',
 '_peek',
 '_populate_data_from_system',
 '_query',
 '_server',
 '_system',
 '_update',
 '_upsert',
 '_validate_tenant_database',
 'clear_system_cache',
 'count_collections',
 'create_collection',
 'database',
 'delete_collection',
 'from_system',
 'get_collection',
 'get_or_create_collection',
 'get_settings',
 'get_version',
 'heartbeat',
 'list_collections'

In [80]:
dir(client)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add',
 '_admin_client',
 '_count',
 '_create_system_if_not_exists',
 '_delete',
 '_get',
 '_get_identifier_from_settings',
 '_identifer_to_system',
 '_identifier',
 '_modify',
 '_peek',
 '_populate_data_from_system',
 '_query',
 '_server',
 '_system',
 '_update',
 '_upsert',
 '_validate_tenant_database',
 'clear_system_cache',
 'count_collections',
 'create_collection',
 'database',
 'delete_collection',
 'from_system',
 'get_collection',
 'get_or_create_collection',
 'get_settings',
 'get_version',
 'heartbeat',
 'list_collections'

In [82]:
dir(client.get_collection(name="langchain"))

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_a

In [88]:
client.delete_collection(name="langchain")

In [89]:
client.list_collections()

[Collection(name=pop_psychology),
 Collection(name=roo),
 Collection(name=streamlit),
 Collection(name=rag_llm)]

In [39]:
from package import embeddings

In [45]:
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        # embed the documents somehow
        return embeddings

In [62]:
emb = MyEmbeddingFunction(embeddings)

collection = client.get_collection(name='roo', embedding_function=emb)
collection

Collection(name=roo)

In [68]:
dir(embeddings.embed_documents)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [76]:
from package.utils import list_db_collections

In [77]:
list_db_collections()

['pop_psychology', 'roo', 'langchain', 'rag_llm']

In [64]:
emb(["FOO"])

ValueError: Expected embeddings to be a list, got base_url='http://localhost:11434' model='mistral' embed_instruction='passage: ' query_instruction='query: ' mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None show_progress=False headers=None model_kwargs=None

In [51]:
dir(collection)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_a

In [55]:
dir(embeddings)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '__weakref__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose

In [56]:
embeddings.embed_documents(["foo"])

[[-1.4971281290054321,
  2.565671920776367,
  -2.8236083984375,
  -6.327280521392822,
  -4.457996845245361,
  -2.27748703956604,
  -5.745776176452637,
  6.592594146728516,
  2.7100143432617188,
  4.080917835235596,
  0.5275201797485352,
  6.531179904937744,
  -7.3103837966918945,
  1.855440616607666,
  5.349830627441406,
  0.25641947984695435,
  -0.6482337713241577,
  3.1755452156066895,
  0.19990728795528412,
  -6.779272556304932,
  8.942420959472656,
  -5.537298679351807,
  5.716923713684082,
  5.040398120880127,
  -8.94235610961914,
  1.7492998838424683,
  -1.316156029701233,
  -2.185901165008545,
  1.1821198463439941,
  -2.6535887718200684,
  -1.7652981281280518,
  14.186317443847656,
  -3.284661293029785,
  -0.6676474809646606,
  3.4080379009246826,
  0.8923372626304626,
  -1.3582274913787842,
  5.707087993621826,
  -5.280853271484375,
  3.984663486480713,
  2.9445595741271973,
  0.1856198012828827,
  1.9413093328475952,
  -0.44933897256851196,
  5.316654682159424,
  -0.6292682290

In [ ]:
collection

In [ ]:
collection.query("")

In [6]:
from package.crud.read_vectordb import load_vectordb
from package.utils import get_context, format_context, generate_prompt

In [91]:
query = "do you know uncle chub?"

collections = ['roo', 'mynotes']

contexts = []
for collection in collections:
    vectordb = load_vectordb(collection_name=collection)
    context = get_context(query, vectordb)
    context = format_context(context)
    contexts.append(context)

In [93]:
context

'Uncle Chub has many kids. He loves taking care of his guild members.\n\nUncle Chub was retired from the game, Ragnarok Online Origin. And he continues living a good life.\n\nUncle Chub is AWESOME!!\n\nUncle Chub has many good friends both in Thai and abroad'

In [92]:
contexts

['A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.\n\nMilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.\n\nRagnarok Online Origin or so called ROO is still open for the global service.\n\nUncle chub is a former guild leader of JointBoi from the game called ROO',
 'Uncle Chub has many kids. He loves taking care of his guild members.\n\nUncle Chub was retired from the game, Ragnarok Online Origin. And he continues living a good life.\n\nUncle Chub is AWESOME!!\n\nUncle Chub has many good friends both in Thai and abroad']

In [94]:
"\n\n".join(contexts)

'A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.\n\nMilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.\n\nRagnarok Online Origin or so called ROO is still open for the global service.\n\nUncle chub is a former guild leader of JointBoi from the game called ROO\n\nUncle Chub has many kids. He loves taking care of his guild members.\n\nUncle Chub was retired from the game, Ragnarok Online Origin. And he continues living a good life.\n\nUncle Chub is AWESOME!!\n\nUncle Chub has many good friends both in Thai and abroad'

In [95]:
print(generate_prompt(query, "\n\n".join(contexts)))

Query: do you know uncle chub?

Context: A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.

MilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.

Ragnarok Online Origin or so called ROO is still open for the global service.

Uncle chub is a former guild leader of JointBoi from the game called ROO

Uncle Chub has many kids. He loves taking care of his guild members.

Uncle Chub was retired from the game, Ragnarok Online Origin. And he continues living a good life.

Uncle Chub is AWESOME!!

Uncle Chub has many good friends both in Thai and abroad


In [2]:
from package.query_context import query_single_collection, query_multiple_collections

C:\Users\doublebank\.conda\envs\local-rag-llm\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [3]:
query = "do you know uncle chub?"

collections = ['roo', 'mynotes']

In [7]:
context = query_single_collection(query, collection=collections[0]))

'Query: do you know uncle chub?\n\nContext: A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.\n\nMilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.\n\nRagnarok Online Origin or so called ROO is still open for the global service.\n\nUncle chub is a former guild leader of JointBoi from the game called ROO'

In [14]:
print(generate_prompt(query, context))

Query: do you know uncle chub?

Context: A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.

MilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.

Ragnarok Online Origin or so called ROO is still open for the global service.

Uncle chub is a former guild leader of JointBoi from the game called ROO


In [20]:
contexts = query_multiple_collections(query, collections=collections)

In [21]:
print(contexts)

A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.

MilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.

Ragnarok Online Origin or so called ROO is still open for the global service.

Uncle chub is a former guild leader of JointBoi from the game called ROO

Uncle Chub has many kids. He loves taking care of his guild members.

Uncle Chub was retired from the game, Ragnarok Online Origin. And he continues living a good life.

Uncle Chub is AWESOME!!

Uncle Chub has many good friends both in Thai and abroad


In [22]:
print(generate_prompt(query, contexts))

Query: do you know uncle chub?

Context: A classic mmorpg game with the new top-up systems will drain all your money out of the pocket. This game will dry you out, so watch out.

MilkyWay is a guild from the well-known game called Ragnarok Online Origin. The former guild leader is UncleChub, but now the guild leader is Wennedy.

Ragnarok Online Origin or so called ROO is still open for the global service.

Uncle chub is a former guild leader of JointBoi from the game called ROO

Uncle Chub has many kids. He loves taking care of his guild members.

Uncle Chub was retired from the game, Ragnarok Online Origin. And he continues living a good life.

Uncle Chub is AWESOME!!

Uncle Chub has many good friends both in Thai and abroad


In [34]:
from langchain_community.llms import Ollama
llm = Ollama(base_url = 'http://localhost:11434', model="mistral")

In [35]:
llm.invoke("hi there")

" Hello! How can I help you today? If you have any questions or need assistance with something, feel free to ask. I'm here to help. If you just wanted to say hi, then hey there yourself! :)"

In [26]:
print(
    llm.invoke("Write me a joke about Batman and Robin")
)

 Why did Batman take Robin to the doctor?

Because he had the flu, but Batman didn't want to let him Bat-rest!

(I hope that brought a smile to your face!)


In [27]:
dir(llm)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [28]:
llm.invoke()

In [33]:
import os
print(os.getenv('OLLAMA_SERVER_URL'))

None


In [29]:
import ollama
ollama.chat()

RequestError: must provide a model

In [30]:
dir(ollama)

['AsyncClient',
 'ChatResponse',
 'Client',
 'GenerateResponse',
 'Message',
 'Options',
 'ProgressResponse',
 'RequestError',
 'ResponseError',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_client',
 '_types',
 'chat',
 'copy',
 'create',
 'delete',
 'embeddings',
 'generate',
 'list',
 'pull',
 'push',
 'show']

ollama._types.Options